In [2]:
from google.auth import compute_engine
from google.cloud.container_v1 import ClusterManagerClient
from kubernetes import client


In [3]:
import os
import dotenv

In [31]:
def setup_gke():
    dotenv.load_dotenv()
    project_id = os.getenv("GKE_PROJECT_ID")
    zone = os.getenv("GKE_ZONE")
    cluster_id = os.getenv("GKE_CLUSTER_ID")

    credentials = compute_engine.Credentials()

    cluster_manager_client = ClusterManagerClient(credentials=credentials)
    cluster = cluster_manager_client.get_cluster(name=f'projects/{project_id}/locations/{zone}/clusters/{cluster_id}')

    configuration = client.Configuration()
    configuration.host = f"https://{cluster.endpoint}:443"
    configuration.verify_ssl = False
    configuration.api_key = {"authorization": "Bearer " + credentials.token}
    client.Configuration.set_default(configuration)

In [32]:
setup_gke()
v1 = client.CoreV1Api()
print("Listing pods with their IPs:")
pods = v1.list_pod_for_all_namespaces(watch=False)
for i in pods.items:
    print("%s\t%s\t%s" % (i.status.pod_ip, i.metadata.namespace, i.metadata.name))

TransportError: Failed to retrieve http://metadata.google.internal/computeMetadata/v1/universe/universe_domain from the Google Compute Engine metadata service. Compute Engine Metadata server unavailable

In [24]:
import os
import google.auth
import base64

from google.cloud.container_v1 import ClusterManagerClient
from kubernetes import client,config
from ruamel.yaml import YAML


In [28]:
def setup_gke_2():  
  PROJECT = os.getenv("GKE_PROJECT_ID")
  ZONE = os.getenv("GKE_ZONE")
  CLUSTER = os.getenv("GKE_CLUSTER_ID")

  # Get Application Default Credentials
  # `project_id` is the Service Account's
  # This may differ to the cluster's `PROJECT`
  credentials, project_id = google.auth.default()

  # Get the cluster config from GCP
  cluster_manager_client = ClusterManagerClient(credentials=credentials)

  name=f"projects/{PROJECT}/locations/{ZONE}/clusters/{CLUSTER}"
  cluster = cluster_manager_client.get_cluster(name=name)

  SERVER = cluster.endpoint
  CERT = cluster.master_auth.cluster_ca_certificate

  configuration = client.Configuration()

  # Create's a `kubectl` config
  NAME="freddie" # arbitrary
  CONFIG=f"""
apiVersion: v1
kind: Config
clusters:
- name: {NAME}
  cluster:
    certificate-authority-data: {CERT}
    server: https://{SERVER}
contexts:
- name: {NAME}
  context:
    cluster: {NAME}
    user: {NAME}
current-context: {NAME}
users:
- name: {NAME}
  user:
    auth-provider:
      name: gcp
      config:
        scopes: https://www.googleapis.com/auth/cloud-platform
  """

  # The Python SDK doesn't directly support providing a dict
  # See: https://github.com/kubernetes-client/python/issues/870
  yaml = YAML(typ='safe', pure=True)
  kubeconfig = yaml.load(CONFIG)

  loader = config.kube_config.KubeConfigLoader(kubeconfig)
  loader.load_and_set(configuration)

  config.load_kubernetes_config()

In [29]:
setup_gke_2()

AttributeError: module 'kubernetes.config' has no attribute 'load_kubernetes_config'

In [30]:
config.load_kube_config()
v1 = client.CoreV1Api()
print("Listing pods with their IPs:")
pods = v1.list_pod_for_all_namespaces(watch=False)
for i in pods.items:
    print("%s\t%s\t%s" % (i.status.pod_ip, i.metadata.namespace, i.metadata.name))

ERROR:root:[Errno 2] No such file or directory: 'gke-gcloud-auth-plugin'


Listing pods with their IPs:


ApiException: (403)
Reason: Forbidden
HTTP response headers: HTTPHeaderDict({'Audit-Id': '153e8fd3-3b9c-4c2d-a21e-1014a4a23883', 'Cache-Control': 'no-cache, private', 'Content-Type': 'application/json', 'X-Content-Type-Options': 'nosniff', 'X-Kubernetes-Pf-Flowschema-Uid': '1cf2ce27-4108-4c16-81cc-cca9f2641e39', 'X-Kubernetes-Pf-Prioritylevel-Uid': '01fed44f-65da-4409-b4c0-0b88105b353d', 'Date': 'Sun, 05 May 2024 05:07:19 GMT', 'Content-Length': '253'})
HTTP response body: {"kind":"Status","apiVersion":"v1","metadata":{},"status":"Failure","message":"pods is forbidden: User \"system:anonymous\" cannot list resource \"pods\" in API group \"\" at the cluster scope","reason":"Forbidden","details":{"kind":"pods"},"code":403}

